In [ ]:
import pandas as pd 


to do:

    - create dataframe with previous and current district maps with 4 districts
    - create gpd dataframes with shapef_ia_redist equivalent for above maps
    - compute compactness scores for above maps
    - export all scores to a csv in order to compare to simulated maps' compactness scores
